<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Validate_Classifier_Timing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
from glob import glob
import csv
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

gesamtabweichung = 0
gesamtschwünge = 0

# ─── 2. Turn-Klassifikationsmodell ────────────────────────────────────────────
class FrameWiseLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size,
                            num_layers=2, bidirectional=True,
                            batch_first=True, dropout=0.3)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(self.dropout(out))

# Modell laden
model_turn = torch.load("left_right_classifier.pt", map_location="cpu")
model_turn.eval()


files = glob("*features.pkl")

for file in files:
  with open(file, "rb") as f:
    features = pickle.load(f)

  # Extrahiere Frame-Indizes
  frame_indices = sorted([f for f in features["COM_to_ground"].keys() if isinstance(f, int)])

  # Baue die Sequenz für die Turn-Klassifikation auf
  seq_turn = []
  for i in frame_indices:
      try:
          v = [
              features["COM_to_ground"][i],
              features["knee_angles_right"][i],
              features["knee_angles_left"][i]
          ]

          # Füge joint_angles hinzu
          for axis in features["joint_angles"][i]:
              v.extend(features["joint_angles"][i][axis].values())

          # Füge axis_angles hinzu
          for axis in features["axis_angles"][i]:
              v.extend(features["axis_angles"][i][axis].values())

          # Füge COM_angles hinzu
          v.extend(features["COM_angles"][i].values())

          # Füge style_idx als zusätzliches Feature hinzu
          v.append(style_idx)

          seq_turn.append(v)
      except KeyError as e:
          print(f"Fehlendes Feature in Frame {i}: {e}")
          continue

  if not seq_turn:
      raise RuntimeError("Turn-Sequence ist leer – bitte Features prüfen!")

  # Konvertiere zu Tensor
  arr_turn = np.array(seq_turn, dtype=np.float32)
  x_turn = torch.tensor(arr_turn).unsqueeze(0).float()  # [1, n_frames, feat_dim+1]

  # Vorhersage treffen
  with torch.no_grad():
      logits = model_turn(x_turn)
      preds = logits.argmax(dim=-1).squeeze().numpy()

  # Labels erstellen
  labels_turn = ["Linksschwung" if p == 1 else "Rechtsschwung" for p in preds]
  print("Schwung pro Frame:", labels_turn)




  abweichung = 0
  schwünge = 0
  prevValue = 0

  for index, real_val in features["schwung_labels"].items():
    if (prevValue == real_val):
      continue
    else:
      schwünge += 1
      if (real_val == preds[index]):
        # wenn der echte Wert der prediction entspricht, dann war Schwungwechsel entweder gleich oder zuvor von Modell erkannt
        counter = index
        while counter > 0:
          if (preds[counter] != real_val):
            # wenn der vorherige Wert sich vom momentanten unterscheidet, dann hat das Modell den Schwungwechsel auch genau auf diesen Frame vorhergesagt (Modell & Realität gleich)
            break
          else:
            abweichung += 1
            counter -= 1
      else:
        # in diesem Fall entspricht der reale Wert nicht dem Wert der Prediction, d.h. die Prediction ändert ihren Wert erst zu einem späteren Wert
        counter = index
        while counter <= len(preds):
          if (preds[counter] == real_val):
            # sobald die prediction mit dem realen Wert übereinstimmt, bricht man aus der Schleife
            break
          else:
            # solange der prediction Wert nicht mit dem realen übereinstimmt, wird die Abweichung erhöht und der counter ebenfalls
            abweichung += 1
            counter += 1
    prevValue = real_val

  print(abweichung)
  print(schwünge)
  print("Durchschnittliche Abweichung in Frames: ", abweichung/schwünge)
  print("Durchschnittliche Abweichung in Sekunden: ", (abweichung/schwünge)/25)

  gesamtschwünge += schwünge
  gesamtabweichung += abweichung


print("Durchschnittliche Gesamtabweichung in Frames: ", gesamtabweichung/gesamtschwünge)
print("Durchschnittliche Gesamtabweichung in Sekunden: ", (gesamtabweichung/gesamtschwünge)/25)

Schwung pro Frame: ['Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung',

In [52]:
# Hier der Code, um nur ein einziges File anzuschauen

abweichung = 0
schwünge = 0
prevValue = 0

for index, real_val in features["schwung_labels"].items():
  if (prevValue == real_val):
    continue
  else:
    schwünge += 1
    if (real_val == preds[index]):
      # wenn der echte Wert der prediction entspricht, dann war Schwungwechsel entweder gleich oder zuvor von Modell erkannt
      counter = index
      while counter > 0:
        if (preds[counter] != real_val):
          # wenn der vorherige Wert sich vom momentanten unterscheidet, dann hat das Modell den Schwungwechsel auch genau auf diesen Frame vorhergesagt (Modell & Realität gleich)
          break
        else:
          abweichung += 1
          counter -= 1
    else:
      # in diesem Fall entspricht der reale Wert nicht dem Wert der Prediction, d.h. die Prediction ändert ihren Wert erst zu einem späteren Wert
      counter = index
      while counter <= len(preds):
        if (preds[counter] == real_val):
          # sobald die prediction mit dem realen Wert übereinstimmt, bricht man aus der Schleife
          break
        else:
          # solange der prediction Wert nicht mit dem realen übereinstimmt, wird die Abweichung erhöht und der counter ebenfalls
          abweichung += 1
          counter += 1
  prevValue = real_val

print(abweichung)
print(schwünge)
print("Durchschnittliche Abweichung in Frames: ", abweichung/schwünge)
print("Durchschnittliche Abweichung in Sekunden: ", (abweichung/schwünge)/25)

19
11
Durchschnittliche Abweichung in Frames:  1.7272727272727273
Durchschnittliche Abweichung in Sekunden:  0.06909090909090909
